In [1]:
import cv2 as cv
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

In [2]:
def calibrate_camera(images_folder):
    images_names = sorted(glob.glob(images_folder))
    images = []
    for imname in images_names:
        im = cv.imread(imname, 1)
        images.append(im)
 
    #criteria used by checkerboard pattern detector.
    #Change this if the code can't find the checkerboard
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
    rows = 6 #number of checkerboard rows.
    columns = 9 #number of checkerboard columns.
    world_scaling = 24. #change this to the real world square size. Or not.
 
    #coordinates of squares in the checkerboard world space
    objp = np.zeros((rows*columns,3), np.float32)
    objp[:,:2] = np.mgrid[0:rows,0:columns].T.reshape(-1,2)
    objp = world_scaling* objp
 
    #frame dimensions. Frames should be the same size.
    width = images[0].shape[1]
    height = images[0].shape[0]
 
    #Pixel coordinates of checkerboards
    imgpoints = [] # 2d points in image plane.
 
    #coordinates of the checkerboard in checkerboard world space.
    objpoints = [] # 3d point in real world space
 
 
    for frame in images:
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
 
        #find the checkerboard
        ret, corners = cv.findChessboardCorners(gray, (rows, columns), None)
        print(ret)
        if ret == True:
 
            #Convolution size used to improve corner detection. Don't make this too large.
            conv_size = (11, 11)
 
            #opencv can attempt to improve the checkerboard coordinates
            corners = cv.cornerSubPix(gray, corners, conv_size, (-1, -1), criteria)
            cv.drawChessboardCorners(frame, (rows,columns), corners, ret)
            cv.imshow('img', frame)
            k = cv.waitKey(500)
 
            objpoints.append(objp)
            imgpoints.append(corners)
 
 
 
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, (width, height), None, None)
    print('rmse:', ret)
    print('camera matrix:\n', mtx)
    print('distortion coeffs:', dist)
    print('Rs:\n', rvecs)
    print('Ts:\n', tvecs)
 
    return mtx, dist

In [3]:
"""mtx3, dist3 = calibrate_camera(images_folder = 'left/*')
mtx4, dist4 = calibrate_camera(images_folder = 'right/*')"""

mtx1 = [[1563.4010009765625, 0.0, 986.95068359375],
        [0.0, 1561.4229736328125, 518.3526611328125],
        [0.0, 0.0, 1.0]]
mtx1 = np.array(mtx1)
mtx2 = [[1564.150634765625, 0.0, 954.2699584960938],
        [0.0, 1562.171630859375, 539.9055786132813],
        [0.0, 0.0, 1.0]]
mtx2 = np.array(mtx2)

dist = [0, 0, 0, 0, 0]
dist1 = np.array(dist)
dist2 = np.array(dist)

In [10]:
cv.destroyAllWindows()

In [5]:
def stereo_calibrate(mtx1, dist1, mtx2, dist2, frames_folder):
    #read the synched frames
    images_names = glob.glob(frames_folder)
    images_names = sorted(images_names)
    c1_images_names = images_names[:len(images_names)//2]
    c2_images_names = images_names[len(images_names)//2:]
 
    c1_images = []
    c2_images = []
    for im1, im2 in zip(c1_images_names, c2_images_names):
        _im = cv.imread(im1, 1)
        c1_images.append(_im)
 
        _im = cv.imread(im2, 1)
        c2_images.append(_im)
 
    #change this if stereo calibration not good.
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
 
    rows = 6 #number of checkerboard rows.
    columns = 9 #number of checkerboard columns.
    world_scaling = 24. #change this to the real world square size. Or not.
 
    #coordinates of squares in the checkerboard world space
    objp = np.zeros((rows*columns,3), np.float32)
    objp[:,:2] = np.mgrid[0:rows,0:columns].T.reshape(-1,2)
    objp = world_scaling* objp
 
    #frame dimensions. Frames should be the same size.
    width = c1_images[0].shape[1]
    height = c1_images[0].shape[0]
 
    #Pixel coordinates of checkerboards
    imgpoints_left = [] # 2d points in image plane.
    imgpoints_right = []
 
    #coordinates of the checkerboard in checkerboard world space.
    objpoints = [] # 3d point in real world space
 
    for frame1, frame2 in zip(c1_images, c2_images):
        gray1 = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
        gray2 = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
        c_ret1, corners1 = cv.findChessboardCorners(gray1, (rows, columns), None)
        c_ret2, corners2 = cv.findChessboardCorners(gray2, (rows, columns), None)
        print(c_ret1, c_ret2)
        if c_ret1 == True and c_ret2 == True:
            corners1 = cv.cornerSubPix(gray1, corners1, (11, 11), (-1, -1), criteria)
            corners2 = cv.cornerSubPix(gray2, corners2, (11, 11), (-1, -1), criteria)
 
            cv.drawChessboardCorners(frame1, (rows, columns), corners1, c_ret1)
            cv.imshow('img', frame1)
 
            cv.drawChessboardCorners(frame2, (rows, columns), corners2, c_ret2)
            cv.imshow('img2', frame2)
            k = cv.waitKey(500)
 
            objpoints.append(objp)
            imgpoints_left.append(corners1)
            imgpoints_right.append(corners2)
 
    stereocalibration_flags = cv.CALIB_FIX_INTRINSIC
    ret, CM1, dist1, CM2, dist2, R, T, E, F = cv.stereoCalibrate(objpoints, imgpoints_left, imgpoints_right, mtx1, dist1,
                                                                 mtx2, dist2, (width, height), criteria = criteria, flags = stereocalibration_flags)
 
    print(ret)
    return R, T

In [6]:
R, T = stereo_calibrate(mtx1, dist1, mtx2, dist2, 'sy/*')
#R_, T_ = cv.stereo_calibrate(mtx3, dist3, mtx4, dist4, 'sy/*')
print(R,'R_')
print(T,'T_')

True True
True True
True True
True True
0.8753560923376033
[[ 0.99928268  0.00872328  0.03685158]
 [-0.01073151  0.9984477   0.05465361]
 [-0.03631762 -0.05500988  0.99782511]] R_
[[178.77704545]
 [-20.69556637]
 [ -4.32033718]] T_


In [7]:
cv.destroyAllWindows()

In [8]:
#RT matrix for C1 is identity.
RT1 = np.concatenate([np.eye(3), [[0],[0],[0]]], axis = -1)
P1 = mtx1 @ RT1 #projection matrix for C1
#P1_ = mtx3 @ RT1
#RT matrix for C2 is the R and T obtained from stereo calibration.
RT2 = np.concatenate([R, T], axis = -1)
P2 = mtx2 @ RT2 #projection matrix for C2
#RT2_ = np.concatenate([R_, T_], axis = -1)
#P2_ = mtx4 @ RT2_ #projection matrix for C2
print(P1)
#print(P1_)
print(P2)
#print(P2_)

[[1.56340100e+03 0.00000000e+00 9.86950684e+02 0.00000000e+00]
 [0.00000000e+00 1.56142297e+03 5.18352661e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]
[[ 1.52837182e+03 -3.88497432e+01  1.00983595e+03  2.75511461e+05]
 [-3.63725479e+01  1.53004654e+03  6.24109655e+02 -3.46626008e+04]
 [-3.63176191e-02 -5.50098753e-02  9.97825107e-01 -4.32033718e+00]]


In [9]:
np.savetxt('P1.dat', P1, delimiter=',')
np.savetxt('P2.dat', P2, delimiter=',')
#np.savetxt('P1_.dat', P1_, delimiter=',')
#np.savetxt('P2_.dat', P2_, delimiter=',')